### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.4.1'

### Collect Data

In [28]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [7]:
df1 = pd.read_csv('/content/drive/MyDrive/bank.csv')

In [9]:
df1.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
df1.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


### Check all columns in the dataset

In [11]:
df1.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [13]:
df1.shape

(10000, 14)

In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [17]:
df1['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

Drop the columns which are unique for all users like IDs.

RowNumber, CustomerId and Surname are unique.




In [19]:
df1 = df1.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


The Feature and Target Sets

In [32]:
#Defining X and y, the feature and target sets

X = df1.iloc[:,0:10].values # Credit Score through Estimated Salary
y = df1.iloc[:,10].values # Exited

In [33]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [34]:
y

array([1, 0, 1, ..., 1, 1, 0])

In [35]:
print(X.shape)
print(y.shape)

(10000, 10)
(10000,)


Geography: 3 categories: 'France', 'Spain' and 'Germany'

Gender: 2 categories: 'Male','Female'

Encoding categorical (string based) data. This will convert those strings into scalar values for analysis

In [36]:
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

# The Problem here is that we are treating the countries as one variable with ordinal values (0 < 1 < 2). 
# Therefore, one way to get rid of that problem is to split the countries into respective dimensions.
# Using One-Hot Encoding with the help of ColumnTransformer from sklearn.compose

transformer = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [1])], remainder='passthrough' )

# Gender does not need this as it is binary

X = transformer.fit_transform(X.tolist())
X = X.astype('float64')


In [37]:
X.shape

(10000, 12)

In [40]:
X

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [41]:
# A '0' on two countries means that the country has to be the one variable which wasn't included 
# This will save us from the problem of using too many dimensions
X = X[:,1:] # Got rid of Spain as a dimension.

In [43]:
X.shape

(10000, 11)

In [46]:
## Spliting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state = 2)

In [47]:
# Feature Scaling 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


Building the neural net

In [132]:
# Initializing the ANN
classifier = Sequential()

In [133]:
# The amount of nodes (dimensions) in hidden layer should be the average of input and output layers, in this case 6.
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

In [134]:
#Add 1st hidden layer
classifier.add(Dense(6, activation='sigmoid', kernel_initializer='uniform'))

In [135]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
classifier.add(Dense(1, activation = 'sigmoid', kernel_initializer='uniform')) 

In [136]:
# Create optimizer with default learning rate
# Compile the model
classifier.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])

In [137]:
classifier.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 6)                 72        
_________________________________________________________________
dense_45 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [138]:
classifier.fit(X_train, y_train, batch_size = 32, epochs = 100)

Train on 7000 samples
Epoch 1/100
7000/7000 [==============================] - 0s 38us/sample - loss: 0.2111 - accuracy: 0.7926
Epoch 2/100
7000/7000 [==============================] - 0s 33us/sample - loss: 0.1786 - accuracy: 0.7926
Epoch 3/100
7000/7000 [==============================] - 0s 32us/sample - loss: 0.1697 - accuracy: 0.7926
Epoch 4/100
7000/7000 [==============================] - 0s 35us/sample - loss: 0.1666 - accuracy: 0.7926
Epoch 5/100
7000/7000 [==============================] - 0s 32us/sample - loss: 0.1654 - accuracy: 0.7926
Epoch 6/100
7000/7000 [==============================] - 0s 31us/sample - loss: 0.1649 - accuracy: 0.7926
Epoch 7/100
7000/7000 [==============================] - 0s 32us/sample - loss: 0.1647 - accuracy: 0.7926
Epoch 8/100
7000/7000 [==============================] - 0s 32us/sample - loss: 0.1645 - accuracy: 0.7926
Epoch 9/100
7000/7000 [==============================] - 0s 29us/sample - loss: 0.1645 - accuracy: 0.7926
Epoch 10/100
7000/7000 [

In [139]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [140]:
cm1 = confusion_matrix(y_test, y_pred)
print(cm1)
accuracy_model1 = ((cm1[0][0]+cm1[1][1])*100)/(cm1[0][0]+cm1[1][1]+cm1[0][1]+cm1[1][0])
print(accuracy_model1, '% of testing data was classified correctly')

[[2415    0]
 [ 585    0]]
80.5 % of testing data was classified correctly


Tuning the Neural Net 

In [141]:
# Initializing the ANN with 1 Hidden Layer
classifier2 = Sequential()
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier2.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))

In [142]:
# Adding the hidden layer
# Notice that we do not need to specify input dim. 
classifier2.add(Dense(activation = 'relu', units=6, kernel_initializer='uniform')) 

In [143]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
classifier2.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform')) 

In [144]:
classifier2.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [145]:
classifier2.fit(X_train, y_train,           
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32)

Train on 7000 samples, validate on 3000 samples
Epoch 1/100
6112/7000 [=========================>....] - ETA: 0s - loss: 0.5995 - accuracy: 0.7899

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


7000/7000 [==============================] - 1s 74us/sample - loss: 0.5805 - accuracy: 0.7920 - val_loss: 0.4470 - val_accuracy: 0.8050
Epoch 2/100
7000/7000 [==============================] - 0s 47us/sample - loss: 0.4418 - accuracy: 0.7926 - val_loss: 0.4326 - val_accuracy: 0.8050
Epoch 3/100
7000/7000 [==============================] - 0s 43us/sample - loss: 0.4310 - accuracy: 0.7926 - val_loss: 0.4289 - val_accuracy: 0.8050
Epoch 4/100
7000/7000 [==============================] - 0s 49us/sample - loss: 0.4256 - accuracy: 0.7926 - val_loss: 0.4257 - val_accuracy: 0.8050
Epoch 5/100
7000/7000 [==============================] - 0s 46us/sample - loss: 0.4212 - accuracy: 0.7930 - val_loss: 0.4227 - val_accuracy: 0.8213
Epoch 6/100
7000/7000 [==============================] - 0s 52us/sample - loss: 0.4166 - accuracy: 0.8251 - val_loss: 0.4205 - val_accuracy: 0.8257
Epoch 7/100
7000/7000 [==============================] - 0s 47us/sample - loss: 0.4127 - accuracy: 0.8294 - val_loss: 0.4171

In [160]:
# To use the confusion Matrix, we need to convert the probabilities that a customer will
# leave the bank into the form true or false. 
# So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.
y_pred = classifier2.predict(X_test)
y_pred = (y_pred > 0.5)
print(y_pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


In [161]:
cm2 = confusion_matrix(y_test, y_pred)
print(cm2)

[[2330   85]
 [ 405  180]]


In [162]:
accuracy_model2 = ((cm2[0][0]+cm2[1][1])*100)/(cm2[0][0]+cm2[1][1]+cm2[0][1]+cm2[1][0])
print (accuracy_model2, '% of testing data was classified correctly')

83.66666666666667 % of testing data was classified correctly


In [163]:
#Initializing Neural Network with 2 Hidden Layers

model = Sequential()
model.add(Dense(6, input_shape=(11,), activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [164]:
# Compiling Neural Network
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [165]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 6)                 72        
_________________________________________________________________
dense_55 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_56 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 7         
Total params: 163
Trainable params: 163
Non-trainable params: 0
_________________________________________________________________


In [166]:
# Fitting our model 
model.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size = 32, epochs = 100)

Train on 7000 samples, validate on 3000 samples
Epoch 1/100
6176/7000 [=========================>....] - ETA: 0s - loss: 0.6493 - accuracy: 0.6737

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


7000/7000 [==============================] - 1s 81us/sample - loss: 0.6397 - accuracy: 0.6881 - val_loss: 0.5505 - val_accuracy: 0.8050
Epoch 2/100
7000/7000 [==============================] - 0s 46us/sample - loss: 0.5184 - accuracy: 0.7926 - val_loss: 0.4807 - val_accuracy: 0.8050
Epoch 3/100
7000/7000 [==============================] - 0s 52us/sample - loss: 0.4661 - accuracy: 0.8011 - val_loss: 0.4434 - val_accuracy: 0.8173
Epoch 4/100
7000/7000 [==============================] - 0s 50us/sample - loss: 0.4252 - accuracy: 0.8246 - val_loss: 0.4104 - val_accuracy: 0.8277
Epoch 5/100
7000/7000 [==============================] - 0s 52us/sample - loss: 0.3947 - accuracy: 0.8354 - val_loss: 0.3862 - val_accuracy: 0.8370
Epoch 6/100
7000/7000 [==============================] - 0s 50us/sample - loss: 0.3766 - accuracy: 0.8426 - val_loss: 0.3731 - val_accuracy: 0.8467
Epoch 7/100
7000/7000 [==============================] - 0s 51us/sample - loss: 0.3658 - accuracy: 0.8527 - val_loss: 0.3650

To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false. So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.
y_pred = (y_pred > 0.5)
print(y_pred)

In [167]:
# Predicting the Test set results
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
print(y_pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


[[False]
 [ True]
 [False]
 ...
 [False]
 [False]
 [False]]


In [169]:
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[2335,   80],
       [ 325,  260]])

In [170]:
accuracy_model3 = ((cm[0][0]+cm[1][1])*100)/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0])
print (accuracy_model3, '% of testing data was classified correctly')

86.5 % of testing data was classified correctly


The optimised model gives better results as seen from the confusion matrix as well as improved accuracy. 
Compared to single hidden layer, 2 hidden layers provided higher accuracy